# convolution network

## description in textbook

- continous form
  
$\begin{equation*}
(f*g)(n)=∫_{-∞}^{∞}f{τ}g(n-τ)dτ
\end{equation*}$

- discrete from

$\begin{equation*}
(f*g)(n)=∑_{τ=-∞}^{∞}f{τ}g(n-τ)
\end{equation*}$


对卷积这个名词的理解：**所谓两个函数的卷积，本质上就是先将一个函数翻转，然后进行滑动叠加。**

参考[知乎](https://www.zhihu.com/question/22298352)回答。

**瞬时行为的持续性后果**


In [1]:
# 2d convolution

import torch
from torch import nn

x = torch.rand(1, 3, 28, 28)
# parameters 1st: 3, input has 3 channels
# parameters 2nd: 3, output has 3 channels
layer = nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1)
out = layer.forward(x)
print("padding=0", out.size())

# [b,1,28,28] == kernel[3,1,3,3] ==> [1,2,28,28]
out = layer.forward(x)
print("padding=1", out.size())

# convenient way
out = layer(x)  #.__call__
print("convenient", out.size())

# some information
print("layer.weight", layer.weight)
print("layer.bias", layer.bias)


padding=0 torch.Size([1, 3, 28, 28])
padding=1 torch.Size([1, 3, 28, 28])
convenient torch.Size([1, 3, 28, 28])
layer.weight Parameter containing:
tensor([[[[-0.0315,  0.0885,  0.0659],
          [ 0.0216, -0.1016,  0.0011],
          [ 0.1141, -0.0933, -0.1203]],

         [[-0.1121,  0.0020,  0.1166],
          [-0.0271,  0.0315, -0.0622],
          [ 0.1611,  0.0671,  0.1478]],

         [[ 0.1047, -0.0065,  0.1638],
          [ 0.0945, -0.1282, -0.0828],
          [ 0.1281, -0.1003, -0.0865]]],


        [[[-0.1422, -0.1166, -0.0918],
          [-0.0167,  0.1351, -0.1817],
          [-0.0331,  0.1771, -0.1429]],

         [[ 0.1405,  0.0679, -0.0774],
          [-0.1561, -0.0875, -0.0400],
          [-0.0537,  0.0367, -0.1841]],

         [[-0.1595,  0.0247, -0.1373],
          [ 0.1016,  0.0223,  0.0657],
          [-0.0454,  0.1220, -0.1142]]],


        [[[ 0.0137, -0.0186, -0.0910],
          [ 0.1029,  0.1110,  0.1907],
          [ 0.0869,  0.0613,  0.1082]],

         [[ 0.11

In [2]:
# low-level usage

from torch.nn import functional as F
import torch

x = torch.rand(1, 3, 28, 28)
w = torch.rand(16, 3, 5, 5)
b = torch.rand(16)
out = F.conv2d(x, w, b, stride=1, padding=1)
print("out", out.shape)


out torch.Size([1, 16, 26, 26])


### Max pooling & Subsampling

## Batch Normalization


In [3]:
import torch

from torch import nn

x = torch.rand(100, 16, 784)  # 28*28
layer = nn.BatchNorm1d(16)
out = layer(x)
print("layer.running_mean", layer.running_mean)
print("layer.running_var", layer.running_var)


layer.running_mean tensor([0.0499, 0.0499, 0.0502, 0.0501, 0.0500, 0.0501, 0.0502, 0.0499, 0.0499,
        0.0500, 0.0502, 0.0500, 0.0499, 0.0501, 0.0499, 0.0501])
layer.running_var tensor([0.9083, 0.9083, 0.9083, 0.9083, 0.9083, 0.9084, 0.9084, 0.9083, 0.9083,
        0.9083, 0.9084, 0.9083, 0.9083, 0.9083, 0.9084, 0.9083])


## ResNet Implementation

In [4]:
from torch import nn
from torch.nn import functional as F


class ResBlk(nn.Module):
    def __init__(self, ch_in, ch_out):
        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(ch_out)

        self.extra = nn.Sequential()
        if ch_out != ch_in:
            # [b,ch_in,h,w] ==> [b,ch_out,h,w]
            self.extra = nn.Sequential(
                nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=1),
                nn.BatchNorm2d(ch_out))

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.extra(x) + out
        return out
